In [1]:
import rospy
from thr_infrastructure_msgs.srv import GetCarryPose, GetCarryPoseRequest
import tf
import os
import transformations

In [2]:
rospy.init_node('create_grid')

In [4]:
srv = rospy.ServiceProxy('thr/get_carry_pose', GetCarryPose)
rospy.wait_for_service('thr/get_carry_pose')
tfl = tf.TransformListener()

In [5]:
ObjectTGripper = tfl.lookupTransform('shapeo', 'right_gripper', rospy.Time(0))

In [6]:
ObjectTGripper

((-0.029695486115008607, 0.0009715081584448226, 0.07400247201558541),
 (0.6929710587972967,
  0.6134725228368815,
  -0.061746930110658856,
  0.3736708337767544))

## Defintion of functions

In [39]:
def get_handover_location(method):
    success = False
    while not success:
        rospy.sleep(0.1)
        req = GetCarryPoseRequest()
        req.object = '/shapeo'
        req.location = 'star'
        req.method = method
        res = srv(req)
        success = res.success
    # express the result in VREP frame
    VrepTBase = tfl.lookupTransform('vrep_frame', 'base', rospy.Time(0))
    GripperTVrep = tfl.lookupTransform('right_gripper', 'vrep_tip', rospy.Time(0))
    BaseTObject = transformations.pose_to_list(res.object_pose)
    # apply chain rule of transformation
    robot_pose = transformations.multiply_transform(VrepTBase, BaseTObject)
    robot_pose = transformations.multiply_transform(robot_pose, ObjectTGripper)
    robot_pose = transformations.multiply_transform(robot_pose, GripperTVrep)
    return robot_pose

In [40]:
def convert_to_string(pose):
    # create the string of the pose to write in the textfile
    pose_str = ''
    for lists in pose:
        for value in lists:
            pose_str += str(value) + '\t' 
    pose_str += '\n'
    return pose_str

## Create table of hip positions 

In [41]:
grid_corners = []
# get the 4 corners of the grid by asking the human to move between them
for i in range(4):
    rospy.sleep(5)
    grid_corners.append(get_handover_location('reba'))
    grid_corners.append(get_handover_location('relative'))
    os.system('beep')
    rospy.sleep(0.5)
    os.system('beep')

## Pulbish them to check result in Rviz

In [42]:
tfb = tf.TransformBroadcaster()

In [ ]:
rate = rospy.Rate(10)
while not rospy.is_shutdown():
    for i in range(len(grid_corners)):
        tfb.sendTransform(grid_corners[i][0], grid_corners[i][1], rospy.Time.now(),
                          '/vrep/corner/'+str(i), '/vrep_frame')
    rate.sleep()

## Write result to a text file

In [12]:
pose_str = ''
for pose in hand_over_grid[0]:
    pose_str += convert_to_string(pose)
# write the string to the file
with open('/tmp/reba_table_left.txt', 'w') as f:
    f.write(pose_str)

In [13]:
pose_str = ''
for pose in hand_over_grid[1]:
    pose_str += convert_to_string(pose)
# write the string to the file
with open('/tmp/reba_table_right.txt', 'w') as f:
    f.write(pose_str)